<a href="https://colab.research.google.com/github/uprotom/espp/blob/main/eric_stock_explorer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Data preparation and Configuration 


## 1.1 Load data
Load clean combined stock and currency data from a manually prepared csv file stored on github

In [3]:
import pandas as pd
import altair as alt

#bf : base data frame
bf = pd.read_csv("https://raw.githubusercontent.com/uprotom/espp/main/eri_stock_sekpln.csv")

## 1.2 Set configuration
Allow to change the default values of monthly investment, company contribution and Computershare fees

It is also possible to select the stock price type when buying

In [4]:
#@title Modify default values:

cfg = {
}

def storeConfig():
  #@markdown Monthly investment (PLN) : equals approx 250-4166 SEK
  monthlyInvestment = 1000 #@param {type:"slider", min:110, max:1820, step:10}

  #@markdown Company contribution [%]
  companyContribution = 15 #@param {type: "number"}

  #@markdown Computershare buy fee [%]
  buyFee = 0.2  #@param {type: "number"}

  #@markdown Computershare sell fees [% / SEK] (whichever is highest)
  sellFee = 0.25 #@param {type: "number"}
  sellFeeFlat = 150 #@param {type: "number"} 

  #@markdown Which stock price to use when buying (@market open, close, average or high/low)
  buyType = "average"  #@param ['open', 'close', 'high', 'low', 'average']

  cfg['monthlyInvestment'] = monthlyInvestment
  cfg['monthlyInvestmentPostFee'] = (1 - buyFee / 100) * monthlyInvestment
  cfg['companyContribution'] = monthlyInvestment * companyContribution / 100
  cfg['buyFee'] = buyFee / 100
  cfg['sellFee'] = sellFee / 100
  cfg['sellFeeFlat'] = sellFeeFlat
  cfg['buyType'] = buyType

storeConfig()

## 1.3 Basic pre-calculations

Calculate the number of shares than can be bought each month given the declared monthly investment, buy fee, share price and SEK/PLN rate.

Buying date is every 15th day of each month (or first business day just after that date). All other days should remain at 0.

In [5]:
def findBuyDays():
  # pre-fill with number of shares = 0
  bf['buyShares'] = 0
  bf['companyContribution'] = 0

  # calculate/set sellPrices for all days based on the configured price type:
  if cfg['buyType'] == "open":
    bf['buySellPrice'] = bf['Open']
  elif cfg['buyType'] == "close":
    bf['buySellPrice'] = bf['Close']
  elif cfg['buyType'] == "average":
    bf['buySellPrice'] = (bf['Open'] + bf['Close']) / 2
  elif cfg['buyType'] == "high":
    bf['buySellPrice'] = bf['High']
  else:
    bf['buySellPrice'] = bf['Low']

  # set starting data for search loop
  counter = 0
  prevMonth = 12
  foundBuyDate = False

  # iterate through all rows to fill in the number of shares on buy dates
  for index in bf.index:

    # clear buyDate flag and set new latest month at month change
    if bf.loc[index, 'Date'].month != prevMonth:
      prevMonth = bf.loc[index, 'Date'].month
      foundBuyDate = False

    if bf.loc[index, 'Date'].day >= 15:
      if foundBuyDate == False:
        foundBuyDate = True
        counter = counter + 1
        bf.loc[index, 'companyContribution'] = cfg['companyContribution']
        #debug: print("found a new buy date", bf.loc[index, 'Date'])
        bf.loc[index, 'buyShares'] = cfg['monthlyInvestmentPostFee'] / bf.loc[index, 'SekPln'] / bf.loc[index, 'buySellPrice']

  #debug: bf.loc[df['Date'].dt.day == 15]
  #debug: bf.loc[df['buyShares'] > 0]
  print("Found ", counter, " buy days in the dataset")

# parse dates from text format to dates
bf['Date'] = pd.to_datetime(bf['Date'],dayfirst=True)

#df : locate buy dates and create a filtered data frame with only buy dates to speed up calculations
findBuyDays()

df = bf.loc[bf["buyShares"] > 0,["Date","SekPln","buySellPrice","buyShares","companyContribution"]]
df = df.reset_index()

Found  266  buy days in the dataset


# 2. Scenario exploration

In this section we will explore a couple of different approaches to the ESPP. 

The aim is to evaluate their real-life performance based on the historical stock price and currency exchange rates. 

All models will follow the buy dates (set on 15th of every month, or the first following business days).
Models will only differ in when they recommend to sell the stock (some more than once).

The resulting *value* is calculated per model for all starting days in the dataset. Which means we will get at least 265 results per each approach.

*Note: some results might be different depending on the size of monthly investment. e.g. Buy & Sell model from 2.1 has a better return rate for larger values*

## 2.0 Do not invest in ESPP

pros: no risk

cons: no gains

Simplest approach first. Any other model will have to beat this approach to be viable.

Resulting value is calculated as number of buying periods from the starting date to the end of the range.

### 2.0.1 Code

In [6]:
df['valNoInvestment'] = 0

# to simplify calculations iterate in reverse order and take the previous result as input to the next step
# add full monthly investment value on buyDays
for index in df.index[df.buyShares != 0][::-1]:
    df.loc[index, 'valNoInvestment'] = df.loc[min(df.index.max(),index + 1), 'valNoInvestment'] + cfg['monthlyInvestment']


## 2.1 Invest and sell immediately after

pros: 
* very low risk
* still get the company contribution bonus

cons: 
* relatively small gains as the flat fee eats a big part of company contribution

Second simplest approach. Should be better than not participating depending on the monthly investment size.
Any other model will have to beat this one too to be viable.

Resulting value is calculated as sum of investments decreased by the flat sale fee and buy/sell fees.

**Note: transfer fees and currency spread will apply as well, but are not included for any model**

### 2.1.1 Code

In [7]:
def calculateValBuyAndSell():

  df['valBuyAndSell'] = 0

  # use two helper variables, we'll iterate in reverse order and keep increasing both
  # accBuyAndSellInv - accumulated buy&sell cash from this day until the end of data
  # accContribInc - accumulated company contributions from this day until the end of data
  accBuyAndSellInv = 0
  accContribInv = 0

  for index in df.index[::-1]: 
    accBuyAndSellInv += cfg['monthlyInvestmentPostFee'] - cfg['sellFeeFlat'] * df.loc[index, 'SekPln']
    accContribInv += df.loc[index, 'companyContribution']   
    df.loc[index, 'valBuyAndSell'] = accBuyAndSellInv + accContribInv 

calculateValBuyAndSell()

## 2.2 Buy and hold model

pros: 

*   gets a % of company contribution for each stock purchase
*   averages buy price over long term
*   generally successful long term as long as there is some growth

cons: 

*   volatile end result making it hard to figure out when to sell

The buy and hold model is the simplest participation approach. It will not sell any of the aquired stock aiming for a super long-term investment.
Resulting value is calculated as number of shares purchased from given date until the end of data counted at last known price and divided by SEKPLN.

Sell fees are applied.
Company contribution is counted.

### 2.2.1 Code

In [8]:
def calculateValBuyAndHold():

  df['valBuyAndHold'] = 0

  # use two helper variables, we'll iterate in reverse order and keep increasing both
  # accSharesInv - accumulated number of shares from this day until the end of data
  # accContribInc - accumulated company contributions from this day until the end of data
  accSharesInv = 0
  accContribInv = 0

  # sale prices
  finalPrice = df.loc[df.index.max(), 'buySellPrice']
  finalSekPlnRate = df.loc[df.index.max(), 'SekPln']

  for index in df.index[::-1]: 
    accSharesInv = accSharesInv + df.loc[index, 'buyShares']
    accContribInv = accContribInv + df.loc[index, 'companyContribution']
    
    #fee = max(cfg['sellFeeFlat'], cfg['sellFee'] * accSharesInv * finalPrice)
    #df.loc[index, 'valBuyAndHold'] = (accSharesInv * finalPrice - fee) * finalSekPlnRate + accContribInv 
    # don't use fee since we're not selling
    df.loc[index, 'valBuyAndHold'] = (accSharesInv * finalPrice) * finalSekPlnRate + accContribInv 

calculateValBuyAndHold()


### 2.2.2 Plots

2.2.2.1 Long term

In [9]:
# plot
data = df.melt(id_vars =['Date'], value_vars = ['valNoInvestment','valBuyAndHold','valBuyAndSell'])
alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='Date',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy & Hold vs Buy & Sell vs no investment. 2000-2022"
)

alt.Chart(...)

2.2.2.2 Shorter term - 3 years

In [10]:
# plot
data = df[-37:].copy()
data['referenceNoInvestment'] = data['valNoInvestment'] - data['valNoInvestment']
data['compToNIBuyAndHold'] = data['valBuyAndHold'] - data['valNoInvestment']
data['compToNIBuyAndSell'] = data['valBuyAndSell'] - data['valNoInvestment']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceNoInvestment','compToNIBuyAndHold','compToNIBuyAndSell'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy & Hold vs Buy & Sell vs no investment. 2019-2022"
)

alt.Chart(...)

2.2.2.3 Last 12M

In [11]:
# plot

pData = data[-13:].melt(id_vars =['Date'], value_vars = ['referenceNoInvestment','compToNIBuyAndHold','compToNIBuyAndSell'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
     title="Buy & Hold vs Buy & Sell vs no investment as baseline. 2021"
)

alt.Chart(...)

### 2.2.3 Conclusions

No surprise that Buy and sell is a better approach than no participation.

Short term (1Y or less) or in flat/decresing market conditions it can match or beat buy & hold. 
Still - this approach will work only as long as company contribution is larger than buy/sell/transfer fees. Because of this it's easier to do with larger contributions (rerun the computations for e.g 600 / 1200 and 1800 monthly investments)

Buy and hold is still the most effective long term, but then it needs to be compared to other non-ESPP investments - as B&S can provide better overall gains by immediately reinvesting elsewhere. 

## 2.3 Buy and keep for N months

pros: 

*   gets a % of company contribution for each stock purchase
*   averages buy price over a given term
*   lowers sell and transfer fees by selling less frequently 
*   generally successful as long as there is not a big decrease
*   for low values of N cash is not frozen for a long period of time

cons: 

*   risky if sold during a drop in price
*   low gain for short hold periods
*   volatile for longer hold periods

'Buy and keep for N months' model will aggregate the stock during a set time to get full company contribution and then sell regardless of the stock price.

This is used as a benchmark and will be tuned in the following models.

### 2.3.1 Code


In [12]:
def calculateValBuyAndHoldPeriod(buyAndHoldPeriod):

  result = []  
  # we'll iterate in normal order, nested loop, multiple times
  # there has to be a smarter way to do it, but nevermind
  # don't do this at home

  for index in df.index[::1]: 
    # clear helper variables:
    # accSharesSinglePeriod - temporarily accumulated number of shares in the buy and hold period
    # allAccSharesValue - value of shares in all buy and hold periods
    # accContrib - accumulated company contribution
    accSharesSinglePeriod = 0
    allAccSharesValue = 0
    accContrib = 0
    iix = 0
    
    # debug
    #loggerList = []
    #loggerList2 = []

    while index + iix <= df.index.max():
      # buy shares and collect contribution on each step
      accSharesSinglePeriod += df.loc[index + iix, 'buyShares']
      accContrib += cfg['companyContribution']

      #'sell' shares at the end of each hold period
      if (iix % buyAndHoldPeriod) == (buyAndHoldPeriod - 1):
        saleIndex = index + iix
        salePrice = df.loc[saleIndex, 'buySellPrice']
        saleSekPlnRate = df.loc[saleIndex, 'SekPln']
        fee = max(cfg['sellFeeFlat'], cfg['sellFee'] * accSharesSinglePeriod * salePrice)
        allAccSharesValue += (accSharesSinglePeriod * salePrice - fee) * saleSekPlnRate 
        #loggerList2.append((accSharesSinglePeriod * salePrice - fee) * saleSekPlnRate)
        accSharesSinglePeriod = 0
      
      iix += 1
    
    # calculate the value of the remaining shares if there are any at the end
    if accSharesSinglePeriod > 0:
      #'sell' shares and count contribution 
      saleIndex = df.index.max()
      salePrice = df.loc[saleIndex, 'buySellPrice']
      saleSekPlnRate = df.loc[saleIndex, 'SekPln']
      allAccSharesValue += (accSharesSinglePeriod * salePrice) * saleSekPlnRate

    # store
    result.append(allAccSharesValue + accContrib)

  return result

In [13]:
# run calculations for a couple of N 
df['valBuyAndHold3M'] = calculateValBuyAndHoldPeriod(3)
df['valBuyAndHold6M'] = calculateValBuyAndHoldPeriod(6)
df['valBuyAndHold12M'] = calculateValBuyAndHoldPeriod(12)
df['valBuyAndHold24M'] = calculateValBuyAndHoldPeriod(24)
df['valBuyAndHold36M'] = calculateValBuyAndHoldPeriod(36)
df['valBuyAndHold48M'] = calculateValBuyAndHoldPeriod(48)


### 2.3.2 Plots

2.3.2.1 Full period (2000-2022)


In [14]:
# plot
data = df.melt(id_vars =['Date'], value_vars = ['valBuyAndSell','valBuyAndHold','valBuyAndHold3M','valBuyAndHold6M','valBuyAndHold12M','valBuyAndHold24M','valBuyAndHold36M','valBuyAndHold48M'])
alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy and hold periods vs Buy&Sell. 2000-2022"
)

alt.Chart(...)

2.3.2.2 Last 5 years (2017-2022)


In [15]:
# plot
data = df[-61:].copy()
data['referenceBuyAndSell'] = data['valBuyAndSell'] - data['valBuyAndSell']
data['compToBSBuyAndHold'] = data['valBuyAndHold'] - data['valBuyAndSell']
data['compToBSBuyAndHold3M'] = data['valBuyAndHold3M'] - data['valBuyAndSell']
data['compToBSBuyAndHold6M'] = data['valBuyAndHold6M'] - data['valBuyAndSell']
data['compToBSBuyAndHold12M'] = data['valBuyAndHold12M'] - data['valBuyAndSell']
data['compToBSBuyAndHold24M'] = data['valBuyAndHold24M'] - data['valBuyAndSell']
data['compToBSBuyAndHold36M'] = data['valBuyAndHold36M'] - data['valBuyAndSell']
data['compToBSBuyAndHold48M'] = data['valBuyAndHold48M'] - data['valBuyAndSell']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold3M','compToBSBuyAndHold6M','compToBSBuyAndHold12M','compToBSBuyAndHold24M','compToBSBuyAndHold36M','compToBSBuyAndHold48M'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy and hold periods against Buy&Sell reference. 2017-2022"
)

alt.Chart(...)

2.3.2.3 Last 12M

In [16]:
pData = data[-13:].melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold3M','compToBSBuyAndHold6M','compToBSBuyAndHold12M'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy and hold periods against Buy&Sell reference. 2021"
)

alt.Chart(...)

### 2.3.3 Conclusions

For longer investment periods simple Buy and Hold for X models are almost always better than Buy and Sell. They are however too simple and unpredictable, sometimes (rarely!) ending up with worse results if applied at some starting points.

However, in recent years, some of them would have performed better than even Buy and Hold.

3 and 6 month Buy and Holds have consistently performed better than a simple B&S (for all data in long and mid term).
However, Buy & Sell was more effective for investments that are closer to the maximum allowed one.

This means that the following sections will be even more interesting as we're now getting closer to more real-life scenarios.

In [17]:
# plot
data = df[-37:].copy()
data['referenceBuyAndHold'] = data['valBuyAndHold'] - data['valBuyAndHold']
data['compToBH_BS'] = data['valBuyAndSell'] - data['valBuyAndHold']
data['compToBH_BH3M'] = data['valBuyAndHold3M'] - data['valBuyAndHold']
data['compToBH_BH6M'] = data['valBuyAndHold6M'] - data['valBuyAndHold']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceBuyAndHold','compToBH_BS','compToBH_BH3M','compToBH_BH6M'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy and hold periods against Buy&Sell reference. 2017-2022"
)

alt.Chart(...)

## 2.4 Buy and keep until X value

pros: 

*   gets a % of company contribution for each stock purchase
*   averages buy price over a period of time
*   aims to minimize fees paying out in predefined parts

cons: 

*   cash is potentially frozen for a long period of time for larger X
*   unknown time scope of investment

'Buy and keep until X value' model will aggregate the stock until defined value in PLN is reached then sell regardless of the stock price.



### 2.4.1 Code


In [18]:
def calculateValBuyAndHoldUntilValue(targetSellValue):

  result = []
  # we'll again iterate in normal order, nested loop, multiple times
  # good it runs @ google

  for index in df.index[::1]: 
    # clear helper variables:
    # accSharesSinglePeriod - temporarily accumulated number of shares in the buy and hold period
    # allAccSharesValue - value of shares in all buy and hold periods
    # accContrib - company contribution
    accSharesSinglePeriod = 0
    allAccSharesValue = 0
    accContrib = 0
    iix = 0
    
    while index + iix <= df.index.max():
      # gather shares and contribution on each step
      accSharesSinglePeriod += df.loc[index + iix, 'buyShares']
      accContrib += cfg['companyContribution']

      # check current value
      saleIndex = index + iix
      salePrice = df.loc[saleIndex, 'buySellPrice']
      saleSekPlnRate = df.loc[saleIndex, 'SekPln']

      if accSharesSinglePeriod * salePrice * saleSekPlnRate >= targetSellValue:
        #'sell' shares
        fee = max(cfg['sellFeeFlat'], cfg['sellFee'] * accSharesSinglePeriod * salePrice)
        allAccSharesValue += (accSharesSinglePeriod * salePrice - fee) * saleSekPlnRate 
        accSharesSinglePeriod = 0
      
      iix += 1
    
    # add the value of the remaining shares if there are any remaining at the end
    if accSharesSinglePeriod > 0:
      saleIndex = df.index.max()
      salePrice = df.loc[saleIndex, 'buySellPrice']
      saleSekPlnRate = df.loc[saleIndex, 'SekPln']
      allAccSharesValue += accSharesSinglePeriod * salePrice * saleSekPlnRate

    result.append(allAccSharesValue + accContrib)

  return result

In [19]:
# calculate for some predefined values
df['valBuyAndHold5K'] = calculateValBuyAndHoldUntilValue(5000)
df['valBuyAndHold10K'] = calculateValBuyAndHoldUntilValue(10000)
df['valBuyAndHold20K'] = calculateValBuyAndHoldUntilValue(20000)
df['valBuyAndHold40K'] = calculateValBuyAndHoldUntilValue(40000)
df['valBuyAndHold60K'] = calculateValBuyAndHoldUntilValue(60000)
df['valBuyAndHold100K'] = calculateValBuyAndHoldUntilValue(100000)

### 2.4.2 Plots

2.4.2.1 Full period (2000-2022)

In [20]:
# plot
data = df.melt(id_vars =['Date'], value_vars = ['valBuyAndSell','valBuyAndHold','valBuyAndHold5K','valBuyAndHold10K','valBuyAndHold20K','valBuyAndHold40K','valBuyAndHold60K','valBuyAndHold100K'])
alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until X value vs B&S. 2000-2022"
)

alt.Chart(...)

2.3.2.2 Last 5 years (2017-2022)

In [21]:
# plot
data = df[-61:].copy()
data['referenceBuyAndSell'] = data['valBuyAndSell'] - data['valBuyAndSell']
data['compToBSBuyAndHold'] = data['valBuyAndHold'] - data['valBuyAndSell']
data['compToBSBuyAndHold5K'] = data['valBuyAndHold5K'] - data['valBuyAndSell']
data['compToBSBuyAndHold10K'] = data['valBuyAndHold10K'] - data['valBuyAndSell']
data['compToBSBuyAndHold20K'] = data['valBuyAndHold20K'] - data['valBuyAndSell']
data['compToBSBuyAndHold40K'] = data['valBuyAndHold40K'] - data['valBuyAndSell']
data['compToBSBuyAndHold60K'] = data['valBuyAndHold60K'] - data['valBuyAndSell']
data['compToBSBuyAndHold100K'] = data['valBuyAndHold100K'] - data['valBuyAndSell']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold5K','compToBSBuyAndHold10K','compToBSBuyAndHold20K','compToBSBuyAndHold40K','compToBSBuyAndHold60K','compToBSBuyAndHold100K'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until X value against Buy&Sell reference. 2017-2022"
)

alt.Chart(...)

2.4.2.3 Last 12M

In [22]:
pData = data[-13:].melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold5K','compToBSBuyAndHold10K','compToBSBuyAndHold20K'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until X value against Buy&Sell reference. 2021"
)

alt.Chart(...)

## 2.5 Buy and keep until X% increase in value

pros: 

*   gets a % of company contribution for each stock purchase
*   at least X% of return long term as long as there is some growth

cons: 

*   cash is potentially frozen for a long period of time in bad market conditions
*   might miss better results if selling too early
*   unknown time scope of investment

'Buy and keep until X% increase of value' model will aggregate the stock until a defined % increase in value is reached then sell regardless of the stock price.


### 2.5.1 Code

In [23]:
def calculateValBuyAndHoldUntilPercentIncrease(targetPercentIncrease):

  result = []
  # no easy way this time
  # we'll again iterate in normal order, nested loop, multiple times

  for index in df.index[::1]: 
    # clear helper variables:
    # accSharesSinglePeriod - temporarily accumulated number of shares in the buy and hold period
    # accMoneySpentSinglePeriod - investment
    # allAccSharesValue - value of shares sold in all periods
    # accContrib - company contribution
    accSharesSinglePeriod = 0
    accMoneySpentSinglePeriod = 0
    allAccSharesValue = 0
    accContrib = 0
    iix = 0
    
    while index + iix <= df.index.max():
      # gather shares and contribution at each step, count spending
      accSharesSinglePeriod += df.loc[index + iix, 'buyShares']
      accContrib += cfg['companyContribution']
      accMoneySpentSinglePeriod += cfg['monthlyInvestmentPostFee']

      # check current % gain
      saleIndex = index + iix
      salePrice = df.loc[saleIndex, 'buySellPrice']
      saleSekPlnRate = df.loc[saleIndex, 'SekPln']
      currentValue = accSharesSinglePeriod * salePrice * saleSekPlnRate

      currentPercentChange = 100 * (currentValue / accMoneySpentSinglePeriod - 1)

      if  currentPercentChange >= targetPercentIncrease:
        #'sell' shares
        fee = max(cfg['sellFeeFlat'], cfg['sellFee'] * accSharesSinglePeriod * salePrice)
        allAccSharesValue += (accSharesSinglePeriod * salePrice - fee) * saleSekPlnRate 
        accSharesSinglePeriod = 0
        accMoneySpentSinglePeriod = 0
      
      iix += 1
    
    # add the value of shares remaining at the end, no fee (not selling at this point)
    if accSharesSinglePeriod > 0:
      saleIndex = df.index.max()
      salePrice = df.loc[saleIndex, 'buySellPrice']
      saleSekPlnRate = df.loc[saleIndex, 'SekPln']
      allAccSharesValue += accSharesSinglePeriod * salePrice * saleSekPlnRate

    result.append(allAccSharesValue + accContrib)

  return result

In [24]:
# calculate for some predefined values of X
df['valBuyAndHold3Perc'] = calculateValBuyAndHoldUntilPercentIncrease(3)
df['valBuyAndHold5Perc'] = calculateValBuyAndHoldUntilPercentIncrease(5)
df['valBuyAndHold10Perc'] = calculateValBuyAndHoldUntilPercentIncrease(10)
df['valBuyAndHold25Perc'] = calculateValBuyAndHoldUntilPercentIncrease(25)
df['valBuyAndHold50Perc'] = calculateValBuyAndHoldUntilPercentIncrease(50)
df['valBuyAndHold100Perc'] = calculateValBuyAndHoldUntilPercentIncrease(100)

### 2.5.2 Plots


2.5.2.1 Full period (2000-2022)

In [25]:
# plot
data = df.melt(id_vars =['Date'], value_vars = ['valBuyAndSell','valBuyAndHold','valBuyAndHold3Perc','valBuyAndHold5Perc','valBuyAndHold10Perc','valBuyAndHold25Perc','valBuyAndHold50Perc','valBuyAndHold100Perc'])
alt.Chart(data).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until +X% vs B&S. 2000-2022"
)

alt.Chart(...)

Note: Hold until +50% is surprisingly effective in the past of the dataset, also if started closer to present (until 2015 or in H1 2018)

2.5.2.2 Last 5 years (2017-2022)


In [26]:
# plot
data = df[-61:].copy()
data['referenceBuyAndSell'] = data['valBuyAndSell'] - data['valBuyAndSell']
data['compToBSBuyAndHold'] = data['valBuyAndHold'] - data['valBuyAndSell']
data['compToBSBuyAndHold3Perc'] = data['valBuyAndHold3Perc'] - data['valBuyAndSell']
data['compToBSBuyAndHold5Perc'] = data['valBuyAndHold5Perc'] - data['valBuyAndSell']
data['compToBSBuyAndHold10Perc'] = data['valBuyAndHold10Perc'] - data['valBuyAndSell']
data['compToBSBuyAndHold25Perc'] = data['valBuyAndHold25Perc'] - data['valBuyAndSell']
data['compToBSBuyAndHold50Perc'] = data['valBuyAndHold50Perc'] - data['valBuyAndSell']
data['compToBSBuyAndHold100Perc'] = data['valBuyAndHold100Perc'] - data['valBuyAndSell']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold3Perc','compToBSBuyAndHold5Perc','compToBSBuyAndHold10Perc','compToBSBuyAndHold25Perc','compToBSBuyAndHold50Perc','compToBSBuyAndHold100Perc'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until +X% vs B&S. 2017-2022"
)

alt.Chart(...)

2.5.2.3 Last 12M

In [27]:
pData = data[-13:].melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSBuyAndHold3Perc','compToBSBuyAndHold5Perc'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Buy until +X% vs B&S. 2021"
)

alt.Chart(...)

## 2.6 Base for 'smart' scenarios

This model is an attempt to combine several approaches and to use the whole dataset including the stock prices between the buy dates.

In the smart scenario we'll have a set of configurable rules to decide when to sell:
*   hold for at least X buy periods
*   AND don't sell until the price is X% higher than average buy price
*   OR sell immediately if the price is X% higher than the average buy price

This way it'll be a good base to add sell-stop or other approaches.

Narrowing down the simulation to a 5Y period to speed up calculations.
Some good values:

4 1 5

4 0 8

4 0 8

6 -2 10

In [28]:
#@title Modify default values:

sellCfg = {
}

def storeSellConfig():
  #@markdown Minimum period to aggregate the stock before selling [months]
  minHoldPeriod = 6 #@param {type:"slider", min:0, max:24, step:1}

  #@markdown Minimum gain to allow a sale [%]
  minimumGainThr = -2 #@param {type:"slider", min:-10, max:50, step:1}

  #@markdown Threshold for immediate sell (override of holdPeriod)
  immediateSaleThr = 10 #@param {type:"slider", min:1, max:50, step:1}

  sellCfg['minHoldPeriod'] = minHoldPeriod
  sellCfg['minimumGainThr'] = (1 + minimumGainThr / 100)
  sellCfg['immediateSaleThr'] = (1 + immediateSaleThr / 100)

storeSellConfig()

### 2.6.1 Code

In [29]:
intraRangeSales = 0
immediateSales = 0
holdAndGrowthRule = 0
result = []

# TODO: the target is to iterate through the base dataframe (daily data), but for now use df still
for index in df.index[-63::1]: 
  # clear helper variables:
  # accSharesThisPeriod - temporarily accumulated number of shares in the buy and hold period
  # buyPeriodsSinceLastSale
  # accMoneySpentSinglePeriod - investment
  # accAllSoldSharesValue - value of shares sold in all periods
  # accContrib - company contribution
  accSharesThisPeriod = 0
  accMoneySpentSinglePeriod = 0
  accAllSoldSharesValue = 0
  buyPeriodsSinceLastSale = 0
  accContrib = 0
  iix = 0

  while index + iix <= df.index.max():
    # gather shares and contribution on each step, count spending
    buyPeriodsSinceLastSale += 1
    accContrib += cfg['companyContribution']
    accSharesThisPeriod += df.loc[index + iix, 'buyShares']
    accMoneySpentSinglePeriod += cfg['monthlyInvestmentPostFee'] / df.loc[index + iix, 'SekPln']
    
    # precalc so we compare only share prices in the inner loop to speed this up
    avgSharePrice = accMoneySpentSinglePeriod / accSharesThisPeriod

    # check if lookahead through bf is possible
    if index + iix + 1 <= df.index.max():
  
      bfStart = df.loc[index + iix]['index']
      bfEnd = df.loc[index + iix + 1]['index']
  
      # +1 to get an inclusive range
      for bfIx in range(bfStart, bfEnd + 1):
        # calculate current % gain

        # don't use the usual buySellPrice
        # salePrice = bf.loc[bfIx,'buySellPrice']
        # instead compare with High, as we assume the sell order is automated
        maxSalePrice = bf.loc[bfIx,'High']
       
        #currentValue = accSharesThisPeriod * salePrice
        #currentPercentChange = currentValue / accMoneySpentSinglePeriod
        currentPercentChange = maxSalePrice / avgSharePrice
        # check sell conditions
        sellAllowed = False
        
        # check for immediate sale
        if currentPercentChange >= sellCfg['immediateSaleThr']:
          sellAllowed = True
          immediateSales += 1
          # we don't take the highest price but the one that was set in the order
          salePrice = avgSharePrice * sellCfg['immediateSaleThr']

        # check for hold period and minimumGain:
        if buyPeriodsSinceLastSale >= sellCfg['minHoldPeriod']:
          if currentPercentChange >= sellCfg['minimumGainThr']:
            sellAllowed = True
            holdAndGrowthRule += 1
            # same here we take the normal buySell price (usually the average)
            salePrice = bf.loc[bfIx,'buySellPrice']

        if sellAllowed:
          intraRangeSales += 1
          #'sell' shares
          fee = max(cfg['sellFeeFlat'], cfg['sellFee'] * accSharesThisPeriod * salePrice)
          saleSekPlnRate = bf.loc[bfIx, 'SekPln']
          accAllSoldSharesValue += (accSharesThisPeriod * salePrice - fee) * saleSekPlnRate 
          accSharesThisPeriod = 0
          accMoneySpentSinglePeriod = 0
          buyPeriodsSinceLastSale = 0
          #skip rest of the range if we sold
          break
    
    iix += 1
  
  # add the value of shares remaining at the end, skip fee (not selling at this point)
  if accSharesThisPeriod > 0:
    #'sell' shares and count contribution 
    saleIndex = df.index.max()
    salePrice = df.loc[saleIndex, 'buySellPrice']
    saleSekPlnRate = df.loc[saleIndex, 'SekPln']
    accAllSoldSharesValue += accSharesThisPeriod * salePrice * saleSekPlnRate

  df.loc[index, 'valSmartScenario'] = accAllSoldSharesValue + accContrib

print("all intra range sales:", intraRangeSales)
print("number of immediateSales:", immediateSales)
print("number of hold and growth sales:",holdAndGrowthRule)

all intra range sales: 575
number of immediateSales: 394
number of hold and growth sales: 181


### 2.6.2 Plots

In [30]:
# plot
data = df[-61:].copy()
data['referenceBuyAndSell'] = data['valBuyAndSell'] - data['valBuyAndSell']
data['compToBSBuyAndHold'] = data['valBuyAndHold'] - data['valBuyAndSell']
data['compToBSSmartScenario'] = data['valSmartScenario'] - data['valBuyAndSell']
data['compToBSBuyAndHold10K'] = data['valBuyAndHold10K'] - data['valBuyAndSell']

pData = data.melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSSmartScenario','compToBSBuyAndHold10K'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Smart scenario 2017-2022"
)

alt.Chart(...)

In [31]:
# plot
pData = data[-25:].melt(id_vars =['Date'], value_vars = ['referenceBuyAndSell','compToBSBuyAndHold','compToBSSmartScenario','compToBSBuyAndHold10K'])
alt.Chart(pData).mark_line(interpolate='step-after').encode(
    x='yearmonth(Date):T',
    y='value',
    color='variable',
).properties(
    width=1200,
    height=400,
    title="Smart scenario 2021"
)

alt.Chart(...)

# 3. Summary

In [ ]:
# results of short term (2Y) of smart model:
data['compToBSSmartScenario'].describe()

count      61.000000
mean     4572.444614
std      3049.401450
min      -304.288504
25%      1842.924954
50%      4560.906059
75%      7366.821069
max      9235.785153
Name: compToBSSmartScenario, dtype: float64

In [ ]:
# results of short term (1Y) of all models. Sorted:
df[-13:][df.columns[6:]].describe().transpose().sort_values(by=['75%'],ascending=False)

,count,mean,std,min,25%,50%,75%,max
valSmartScenario,13.0,14515.080198,8056.390647,2089.360,8349.072707,14217.892647,20896.145708,27319.443284
valBuyAndSell,13.0,14153.233462,7874.146058,2023.525,8086.120000,14152.645000,20219.050000,26285.575000
valBuyAndHold100Perc,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073
valBuyAndHold50Perc,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073
valBuyAndHold25Perc,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073
valBuyAndHold10Perc,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073
valBuyAndHold5Perc,13.0,14276.553020,7702.778223,2089.360,8349.072707,14405.338376,20103.466086,26308.133773
valBuyAndHold3Perc,13.0,14290.936261,7695.173212,2089.360,8349.072707,14405.338376,20103.466086,26308.133773
valBuyAndHold100K,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073
valBuyAndHold60K,13.0,14225.787585,7664.429760,2089.360,8349.072707,14405.338376,20103.466086,25968.809073


# Random visualizations and checkers

In [ ]:

base = alt.Chart(df.reset_index()).encode(x='Date').properties(
)

alt.layer(
    base.mark_line(color='red').encode(y='SekPln'),
).properties(
    width=1200,
    height=400,
    title="Number of shares that can be bought"
)

NameError: ignored

In [ ]:
base = alt.Chart(df.reset_index()).encode(x='Date').properties(
)

alt.layer(
    base.mark_line(color='orange').encode(y='buyShares'),
).properties(
    width=1200,
    height=400,
    title="Number of shares that can be bought"
)

alt.LayerChart(...)